# Tensorflow Estimators

In [51]:
import tensorflow as tf
import pandas as pd

In [52]:
df = pd.read_csv('../data/iris.csv')

In [53]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [54]:
X = df.drop('species', axis='columns')
y = df['species']

column_names = df.columns[:-1]

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [57]:
y_train.head()

60     versicolor
116     virginica
144     virginica
119     virginica
108     virginica
Name: species, dtype: object

In [58]:
labels = ['setosa', 'versicolor', 'virginica']

In [59]:
feature_columns = [
    tf.feature_column.numeric_column(key=key)
    for key in X_train.columns
]

In [60]:
feature_columns

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Canned Estimators

In [61]:
rm -rf ./models/iris/

In [62]:
model = tf.estimator.DNNClassifier(
    hidden_units=[10, 10], 
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d7e72ee10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
batch_size=32
train_steps=3000

### Input function

In [64]:
def input_fn(features, labels, batch_size, shuffle=True, repeat=None):
    feature_dict = dict(features)

    if labels is None:
        inputs = feature_dict
    else:
        inputs = (feature_dict, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.repeat(repeat)

    return dataset.batch(batch_size)

In [65]:
def train_input_fn():
    return input_fn(X_train, y_train, batch_size)

### Model Training

In [66]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 44.947083, step = 0
INFO:tensorflow:global_step/sec: 191.664
INFO:tensorflow:loss = 4.6415973, step = 100 (0.524 sec)
INFO:tensorflow:global_step/sec: 245.803
INFO:tensorflow:loss = 1.8587031, step = 200 (0.407 sec)
INFO:tensorflow:global_step/sec: 241.025
INFO:tensorflow:loss = 1.2581246, step = 300 (0.418 sec)
INFO:tensorflow:global_step/sec: 243.464
INFO:tensorflow:loss = 4.60306, step = 400 (0.408 sec)
INFO:tensorflow:global_step/sec: 233.461
INFO:tensorflow:loss = 10.793986, step = 500 (0.429 sec)
INFO:tensorflow:global_step/sec: 244.797
INFO:tensorflow:loss = 2.0349557, step = 600 (0.409 sec)
INFO:tensorflow:global_step/sec: 240.803
INFO:tensorflow:l

### Evaluate model on train and test data

In [17]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-23-12:02:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-23-12:02:29
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9809524, average_loss = 0.024422603, global_step = 3000, loss = 0.6410933
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-23-12:02:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018

In [18]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.024422603,
 'loss': 0.6410933,
 'global_step': 3000}

In [19]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.12057793,
 'loss': 2.7130034,
 'global_step': 3000}

### Predict on new data

In [20]:
predict_x = {
    'sepal_length': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_length': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1],
}

In [21]:
expected = ['setosa', 'versicolor', 'virginica']

In [22]:
predictions = model.predict(
    input_fn=lambda:input_fn(predict_x,
                             labels=None,
                             batch_size=batch_size,
                             shuffle=False,
                             repeat=1))

In [23]:
predictions

<generator object Estimator.predict at 0x7f5d06e17e60>

In [24]:
template = ('\nPrediction is "{}", prob: {:.1f}%, expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    pred_label = labels[class_id]

    print(template.format(pred_label, 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "setosa", prob: 100.0%, expected "setosa"

Prediction is "versicolor", prob: 99.9%, expected "versicolor"

Prediction is "virginica", prob: 99.9%, expected "virginica"


In [25]:
pred_dict

{'logits': array([-14.221882  ,  -0.39109015,   6.2040877 ], dtype=float32),
 'probabilities': array([1.3443757e-09, 1.3650779e-03, 9.9863487e-01], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'virginica'], dtype=object)}

## Exercise 1

The `tf.estimator` module offers 2 additional canned classifiers:

- `tf.estimator.BaselineClassifier`
- `tf.estimator.LinearClassifier`

What do they do?

Try substituting them in the model definition and re-run the trainin, evaluation and prediction steps.
Is the performance better or worse?

Note: you can set the logging level of tensorflow with `tf.logging.set_verbosity`

In [34]:
!rm -rf ./models/iris/

In [35]:
model = tf.estimator.BaselineClassifier(
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d05ea5390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
model.train(
    input_fn=train_input_fn,
    steps=3000)
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 259.788
INFO:tensorflow:loss = 34.99904, step = 100 (0.387 sec)
INFO:tensorflow:global_step/sec: 336.44
INFO:tensorflow:loss = 34.378803, step = 200 (0.297 sec)
INFO:tensorflow:global_step/sec: 334.555
INFO:tensorflow:loss = 35.381622, step = 300 (0.299 sec)
INFO:tensorflow:global_step/sec: 320.086
INFO:tensorflow:loss = 34.52932, step = 400 (0.312 sec)
INFO:tensorflow:global_step/sec: 328.189
INFO:tensorflow:loss = 35.75821, step = 500 (0.305 sec)
INFO:tensorflow:global_step/sec: 333.033
INFO:tensorflow:loss = 36.3349, step = 600 (0.302 sec)
INFO:tensorflow:global_step/sec: 331.124
INFO:tensorflow:loss 

In [37]:
train_eval_result

{'accuracy': 0.37142858,
 'average_loss': 1.1013455,
 'loss': 28.91032,
 'global_step': 3000}

In [38]:
test_eval_result

{'accuracy': 0.24444444,
 'average_loss': 1.1533835,
 'loss': 25.951128,
 'global_step': 3000}

In [39]:
!rm -rf ./models/iris/

In [40]:
model = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5d05ea5470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
model.train(
    input_fn=train_input_fn,
    steps=3000)
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 208.15
INFO:tensorflow:loss = 5.6855087, step = 100 (0.482 sec)
INFO:tensorflow:global_step/sec: 276.607
INFO:tensorflow:loss = 3.643992, step = 200 (0.361 sec)
INFO:tensorflow:global_step/sec: 279.711
INFO:tensorflow:loss = 1.8684633, step = 300 (0.358 sec)
INFO:tensorflow:global_step/sec: 268.375
INFO:tensorflow:loss = 1.2505344, step = 400 (0.373 sec)
INFO:tensorflow:global_step/sec: 269.104
INFO:tensorflow:loss = 2.4427412, step = 500 (0.372 sec)
INFO:tensorflow:global_step/sec: 275.696
INFO:tensorflow:loss = 1.7454394, step = 600 (0.363 sec)
INFO:tensorflow:global_step/sec: 278.988
INFO:tensorflow:l

In [42]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.0311858,
 'loss': 0.81862724,
 'global_step': 3000}

In [43]:
test_eval_result

{'accuracy': 0.95555556,
 'average_loss': 0.10346401,
 'loss': 2.3279402,
 'global_step': 3000}

## Exercise 2

Visualize the training results with tensorboard:

- in a terminal, from the notebook folder, run tensorboard with `tensorboard --logdir=models`
- open the browser at [http://localhost:6006](http://localhost:6006)



In [70]:
ls ./models/iris/DNN

checkpoint                        model.ckpt-0.meta
graph.pbtxt                       model.ckpt-3000.data-00000-of-00002
model.ckpt-0.data-00000-of-00002  model.ckpt-3000.data-00001-of-00002
model.ckpt-0.data-00001-of-00002  model.ckpt-3000.index
model.ckpt-0.index                model.ckpt-3000.meta


In [50]:
ls -rf ./models/iris/DNN/checkpoint

./models/iris/DNN/checkpoint
